In [6]:
import numpy as np
import pandas as pd
import os

# File paths
technical_coeff_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/technical_coefficients/technical_coefficients_2019.csv"
price_volatility_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data/II_PI_volatility.csv"
output_dir = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results"

# Dictionary of CPI weight files for each sampling group
cpi_weights_files = {
    "individual": os.path.join(output_dir, "individual_cpi_weights.csv"),
    "core": os.path.join(output_dir, "core_cpi_weights.csv"),
    "periphery": os.path.join(output_dir, "periphery_cpi_weights.csv"),
    "eu28": os.path.join(output_dir, "eu28_and_row_weights")
}

# Load data
unweighted_impacts = pd.read_csv(os.path.join(output_dir, "unweighted_shock_impacts.csv"), index_col=0)
technical_coefficients = pd.read_csv(technical_coeff_path, index_col=0)
price_volatility = pd.read_csv(price_volatility_path, index_col=[0])

# Calculate the Leontief inverse from the technical coefficients
A_transposed = technical_coefficients.T
I = np.eye(A_transposed.shape[0])

# Calculate Leontief inverse
leontief_inverse = pd.DataFrame(
    np.linalg.inv(I - A_transposed.values),
    index=A_transposed.index,
    columns=A_transposed.columns
)

# Function to calculate direct, indirect, and total impacts
def calculate_impacts(cpi_weights, sample_name):
    impacts = []

    for exogenous_sector in unweighted_impacts.columns:
        # Remove the "ip_" prefix if it exists
        if exogenous_sector.startswith('ip_'):
            exogenous_sector = exogenous_sector[3:]  # Remove the prefix
        
        # Extract country code and sector code
        try:
            country_code, sector_code = exogenous_sector.split('_', 1)
        except ValueError:
            print(f"Unexpected format for sector label: {exogenous_sector}")
            continue

        # Retrieve price shock for the exogenous sector
        try:
            price_shock = price_volatility.loc[exogenous_sector, 'price_volatility']
        except KeyError:
            print(f"Price shock not found for sector {exogenous_sector}. Skipping.")
            continue

        # Retrieve CPI weight for the exogenous sector
        try:
            cpi_weight = cpi_weights.loc[exogenous_sector, country_code]
        except KeyError:
            print(f"CPI weight not found for exogenous sector {exogenous_sector} in country {country_code}. Skipping.")
            continue

        # Calculate direct impact
        direct_impact = cpi_weight * price_shock

        # Calculate propagated shocks (\(\sigma_i^E\)) using the Leontief inverse
        propagated_shocks = leontief_inverse[exogenous_sector] * price_shock

        # Calculate indirect impact as the sum of all weighted shocks excluding the exogenous sector
        indirect_impact = (
            (cpi_weights.loc[propagated_shocks.index, country_code] * propagated_shocks)
            .drop(index=exogenous_sector, errors='ignore')
            .sum()
        )

        # Calculate total impact
        total_impact = direct_impact + indirect_impact

        # Store results
        impacts.append({
            'Country': country_code,
            'Sector': sector_code,
            f'{sample_name}_Direct Impact': direct_impact,
            f'{sample_name}_Indirect Impact': indirect_impact,
            f'{sample_name}_Total Impact': total_impact
        })

    # Save impacts for the current sample
    impacts_df = pd.DataFrame(impacts)
    output_file = os.path.join(output_dir, f"{sample_name}_impacts.csv")
    impacts_df.to_csv(output_file, index=False)
    print(f"Impacts for {sample_name} saved to {output_file}")


# Calculate and save impacts for each CPI weight sample
for sample_name, cpi_weights_file in cpi_weights_files.items():
    # Load CPI weights
    cpi_weights = pd.read_csv(cpi_weights_file, index_col=0)

    # Clean up the row index in CPI weights to remove any prefix (e.g., "op_")
    cpi_weights.index = cpi_weights.index.str.replace('^op_', '', regex=True)

    # Calculate impacts
    calculate_impacts(cpi_weights, sample_name)


CPI weight not found for exogenous sector FIGW1_A01 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_A02 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_A03 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_B in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C10-C12 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C13-C15 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C16 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C17 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C18 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C19 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C20 in country FIGW1. Skipping.
CPI weight not found for exogenous sector FIGW1_C21 in country FIGW1. Skipping.
CPI weight not found for exogenous